# Carbon Free Boston

In [ ]:
import emat
emat.__version__

In [ ]:
import os

In [ ]:
import ema_workbench

from ema_workbench import TimeSeriesOutcome, ema_logging, perform_experiments
from ema_workbench.connectors.excel import ExcelModel
from ema_workbench.em_framework.evaluators import MultiprocessingEvaluator, SequentialEvaluator
from ema_workbench import BooleanParameter, IntegerParameter, RealParameter, ScalarOutcome, Constraint

In [ ]:
ema_logging.log_to_stderr(level=ema_logging.INFO)

In [ ]:
excel_file = "./CFB/CFB-Trans_v1.3.3.xlsm"
assert os.path.exists(excel_file)

In [ ]:
scope_file = './CFB/CFB-emat.yaml'
assert os.path.exists(scope_file)

In [ ]:
import os
wd, model_file = os.path.split(excel_file)
wd, model_file

In [ ]:
model = emat.ExcelCoreModel(
    wd=wd, 
    model_file=model_file,
    scope=scope_file,
)

## Using a Database

The exploratory modeling process will typically generate many different sets of outputs,
for different explored modeling scopes, or for different applications.  It is convenient
to organize these outputs in a database structure, so they are stored consistently and 
readily available for subsequent analysis.

The `SQLiteDB` object will create a database to store results.  When instantiated with
no arguments, the database is initialized in-memory, which will not store anything to
disk (which is convenient for this example, but in practice you will generally want to
store data to disk so that it can persist after this Python session ends).

In [ ]:
db = emat.SQLiteDB('./CFB/CFB3.sqldb', initialize=True)

An EMAT Scope can be stored in the database, to provide needed information about what the 
various inputs and outputs represent.

In [ ]:
model.scope.store_scope(db)
model.db = db

Trying to store another scope with the same name (or the same scope) raises a KeyError.

In [ ]:
from emat.experiment.experimental_design import design_experiments

In [ ]:
design = design_experiments(model.scope, db=db, n_samples=500, sampler='lhs')
design.head()

In [ ]:
with MultiprocessingEvaluator(model, n_processes=4) as evaluator:
    lhs_results = model.run_experiments(design_name='lhs', evaluator=evaluator, db=db)
lhs_results.head()

In [ ]:
# model.constraints = [

# Constraint(
#     'Light-duty vehicles: Total Fleet Size', 
#     parameter_names=[
#         "Light-duty vehicles: % of LDV stock: EV",
#         "Light-duty vehicles: % of LDV stock: PHEV",
#     ],
#     function= lambda *x: 100-sum(*x),
# )
# ]

In [ ]:
# model.get_feature_scores('lhs')

In [ ]:
model.db = db

In [ ]:
lhs_results.to_csv('./CFB/CFB-500.csv')

In [ ]:
db

In [ ]:
db.read_experiment_all('CFB','lhs')

In [ ]:
db.read_scope_names()

In [ ]:
db.read_design_names('CFB')

In [ ]:
db.close()

In [ ]:
db.conn.close()